In [1]:
import wandb
wandb.login()


wandb: Currently logged in as: fercos. Use `wandb login --relogin` to force relogin


True

In [2]:
from typing import List, Tuple
from diametery.fiber import Image
from PIL import Image as PILImage
from diametery import __version__ as dy_version
import numpy as np
import json

In [3]:
def create_simple_dataset(sizes:Tuple[int, int, int], img_size:Tuple[int, int], **kwargs):
    """Creates a simple dataset and logs it to wandb.

    Args:
        sizes: Sizes of the train val test splits
        img_size: pixel size of all images.
    """

    with wandb.init(project='diameterY', job_type='load-data') as run:
        raw_data = wandb.Artifact(
            'simple-fibers', type='dataset',
            description='Single straight fibers split into train/val/test',
            metadata={
                'img_size':img_size,
                'sizes':sizes, 
                'dy_version': dy_version
            }
        )
        for set_name, set_size in zip(['train', 'val', 'test'], sizes):
            for i in range(set_size):
                path = f"/home/fer/projects/diameterY/dataset/simple-fibers/{set_name}_{i}"
                image = Image.create(img_size)
                im = image.render_image()
                field_and_weights = image.render_field_and_weights()
                params = image.to_dict()
                pil_im = PILImage.fromarray(im)
                pil_im.save(f"{path}.jpeg")
                raw_data.add_file(f"{path}.jpeg")
                np.savez(
                    f"{path}.npz", 
                    x=im, 
                    y=field_and_weights,
                    d=image.fiber.diameter,
                    p1=image.fiber.p1,
                    p2=image.fiber.p2,
                    bg=image.bg_color,
                    fg=image.fiber.brightness
                )
                raw_data.add_file(f"{path}.npz")
                with open(f'{path}_params.json', 'w') as file:
                    json.dump(params, file, indent=2)
                raw_data.add_file(f'{path}_params.json')
        run.log_artifact(raw_data)


In [4]:
img_size = [224, 224]
create_simple_dataset( [80,10,10], img_size)

AttributeError: 'str' object has no attribute 'write'